In [ ]:
from pathlib import Path
import pandas as pda
import sys
import os
sys.path.append(os.path.join(sys.path[0],'..','..','GithubActivities'))
from DataHandling import DataManager
%load_ext autoreload
%autoreload 2

In [ ]:
print(os.getcwd())
os.chdir("../../")
print(os.getcwd())

In [ ]:
project_folder_name = "./example/"
project_parameter_file = "parameter.yml"

dataHandler = DataManager(project_folder_name + project_parameter_file)

In [ ]:
raw_pdCommits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdCommits.p")
raw_pdUsers = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdUsers.p")
raw_pdReleases = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdReleases.p")
raw_pdEdits = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdEdits.p")
raw_pdIssues = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdIssues.p")
raw_pdBranches = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdBranches.p")
raw_pdPullRequests = dataHandler.unpickle_From_Raw_Data_Folder("raw_pdPullRequests.p")

## Check user list

In [ ]:
github_users_black_list = ["github-classroom[bot]", "actions-user", "web-flow"]
github_users_black_list_ids = raw_pdUsers [raw_pdUsers.login.isin(github_users_black_list)].anonym_uuid.unique().tolist() 
github_users_black_list_ids

In [ ]:
raw_pdCommits['commiter_role'] = "student"
raw_pdCommits.loc[raw_pdCommits.author.isin(github_users_black_list_ids), "commiter_role"] = 'github'

In [ ]:
raw_pdUsers[~raw_pdUsers.anonym_uuid.isin(github_users_black_list_ids)]

## Eliminate none authors

In [ ]:
raw_pdCommits[raw_pdCommits.author.isnull()]

## Anonymize authors and teams

In [ ]:
authors = raw_pdCommits.author.unique()
authors_dict = dict(zip(authors, range(len(authors))))

teams = raw_pdCommits.team.unique()
teams_dict = dict(zip(teams, range(len(teams))))

In [ ]:
def annoymizeDataFrame(raw_df, teams_dict, authors_dict):
    df = raw_df.copy()
    if 'author' in df.columns:
        df['authorKey'] = df['author'].apply(lambda s: authors_dict.get(s) if s in authors_dict else s)
    if 'team' in df.columns:
        df['teamKey'] = df['team'].apply(lambda s: teams_dict.get(s) if s in teams_dict else s)
    return df

anonym_pdCommits = annoymizeDataFrame(raw_pdCommits, teams_dict, authors_dict)
anonym_pdReleases = annoymizeDataFrame(raw_pdReleases, teams_dict, authors_dict)
anonym_pdEdits = annoymizeDataFrame(raw_pdEdits, teams_dict, authors_dict)
anonym_pdIssues = annoymizeDataFrame(raw_pdIssues, teams_dict, authors_dict)
anonym_pdBranches = annoymizeDataFrame(raw_pdBranches, teams_dict, authors_dict)
anonym_pdPullRequests = annoymizeDataFrame(raw_pdPullRequests, teams_dict, authors_dict)

## Save files

In [ ]:
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdCommits, "anonym_pdCommits.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdReleases, "anonym_pdReleases.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdEdits, "anonym_pdEdits.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdIssues, "anonym_pdIssues.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdBranches, "anonym_pdBranches.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(anonym_pdPullRequests, "anonym_pdPullRequests.p")
dataHandler.pickle_File_To_Preproc_Data_Folder(teams_dict, "teams_dict.p")